# Python helper file/script for MATLAB

This cell sets up the notebook to import numpy, datascience, seaborn, pandas, matplotlib etc.

In [20]:
# Run this cell to set up the notebook.

# These lines import the Numpy, Datascience, pandas modules.
import numpy as np
import pandas as pd
import seaborn as sns
from datascience import *
import random
import matplotlib
import matplotlib.pyplot as plt

# Importing plotting libraries and styles
%matplotlib inline
plt.style.use('fivethirtyeight')

# For Pandas to ignore FutureWarning displays
import warnings
warnings.simplefilter('ignore', FutureWarning)

### The cell given below sets up MATLAB for the notebook
Source: https://sehyoun.com/blog/20180904_using-matlab-with-jupyter-notebook.html

In [21]:
import matlab.engine
import io
import scipy.io
from IPython.core.magic import register_cell_magic
ip = get_ipython()

out = io.StringIO()
err = io.StringIO()

# Setup matlab cell magic #
@register_cell_magic
def matlab_magic(line,cell):
    out.truncate(0)
    out.seek(0)
    err.truncate(0)
    err.truncate(0)
    raw = '''{line}.eval("""{cell}""", nargout=0, stdout=out, stderr=err)'''
    ip.run_cell(raw.format(line=line, cell=cell))
    print(out.getvalue())
    print(err.getvalue())
    
# Starting a MATLAB engine called eng
eng = matlab.engine.start_matlab()

In [22]:
%%matlab_magic eng

t = linspace(0,6*pi,100);
plot(sin(t))
grid on
hold on
plot(cos(t), 'r')

In [23]:
%%matlab_magic eng

x = 5;
disp(x);
x

     5


x =

     5





Calling another MATLAB file from Python script

In [24]:
%%matlab_magic eng

my_array = [10, 20, 30, 40, 50]
avgOfThis = averageFunc(my_array)


my_array =

    10    20    30    40    50


avgOfThis =

    30





**Python to MATLAB**

Source: https://stackoverflow.com/questions/10997254/converting-numpy-arrays-to-matlab-and-vice-versa

In [25]:
x = [1,2,3]
y = "hello"
z = [[1,2,3], [4,5,6]]

scipy.io.savemat('test.mat', dict(x=x, y=y, z=z))

In [26]:
%%matlab_magic eng
load test
x
y
z


x =

  1x3 int64 row vector

   1   2   3


y =

    'hello'


z =

  2x3 int64 matrix

   1   2   3
   4   5   6





**MATLAB to Python**

Source: http://www.blogforbrains.com/blog/2014/9/6/loading-matlab-mat-data-in-python

In [27]:
# Similarly we have
#scipy.io.loadmat("averageFunc.m")
variables = scipy.io.loadmat('data.mat', squeeze_me=True)
variables

{'__header__': b'MATLAB 5.0 MAT-file, Platform: MACI64, Created on: Fri Feb  8 01:20:47 2019',
 '__version__': '1.0',
 '__globals__': [],
 'a': array([[1, 2, 3],
        [4, 5, 6]], dtype=uint8),
 'S': array((array([[ 7,  8,  9],
        [10, 11, 12]], dtype=uint8),), dtype=[('b', 'O')]),
 'M': array([(array([[ 2,  4,  6],
        [ 8, 10, 12]], dtype=uint8),),
        (array([[ 1,  3,  5],
        [ 7,  9, 11]], dtype=uint8),)], dtype=[('c', 'O')])}

In [28]:
a = variables['a'] # array
S = variables['S'] # structure containing an array
M = variables['M'] # array of structures

a

array([[1, 2, 3],
       [4, 5, 6]], dtype=uint8)

In [29]:
S

array((array([[ 7,  8,  9],
       [10, 11, 12]], dtype=uint8),), dtype=[('b', 'O')])

In [30]:
M

array([(array([[ 2,  4,  6],
       [ 8, 10, 12]], dtype=uint8),),
       (array([[ 1,  3,  5],
       [ 7,  9, 11]], dtype=uint8),)], dtype=[('c', 'O')])

In [31]:
# Adds the MMB.m as well as MMBOPT1.m and MMBOPT2.m folders to the MATLAB engine path"
eng.addpath(r'/Users/Desktop/monetaryPolicy/mmb-gui-mlab/2.3.2', nargout=0)
eng.addpath(r'/Users/Desktop/monetaryPolicy/mmb-gui-mlab/2.3.2/MMB_OPTIONS', nargout=0)

## **Important:** 
The cell below runs the MMB.m file

In [183]:
eng.MMB(nargout = 0)

### Here, we import the data for 4 IRF (Impulse Response Functions) variables
#### outputgap, inflation, interest, output

In [170]:
irf_4 = pd.read_excel("../mmb-gui-mlab-2.3.2/OUTPUT/results.xls", sheetname = "IRF Mon. Pol. Shock      ")
irf_4 = irf_4.T
irf_headers = irf_4.iloc[0] # grab the first row for the header
irf_4 = irf_4[1:] # take the data less the header row
irf_4_stripped_headers = [myHeader.strip() for myHeader in np.array(irf_headers)] # removing trailing whitespaces
irf_4.columns = irf_4_stripped_headers
modelName = irf_4.columns.values[1]
irf_4 = irf_4.iloc[:, [i for i in range(1, len(irf_4.columns.values), 2)]]
irf_4.columns = ["OutputGap", "Inflation", "Interest", "Output"]
irf_4 = irf_4.reset_index()
irf_4.index.name = "Period"
irf_4.drop('index', axis=1, inplace=True)
print("OUR CURRENT MODEL IS:", modelName)
irf_4.head()

OUR CURRENT MODEL IS: US_ACELm


,OutputGap,Inflation,Interest,Output
Period,,,,
0,0,0,0,0
1,0.0311337,2.86822e-17,1.01557,0.0311337
2,2.18432e-16,-3.81229e-17,3.70087e-16,1.02879e-16
3,2.22032e-16,4.76894e-16,8.16841e-16,-5.71252e-17
4,2.60732e-17,1.9386e-17,9.71128e-18,7.14301e-18


### Here, we import the data for ALL IRF (Impulse Response Functions) variables

In [164]:
old_irf_df = pd.read_excel("../mmb-gui-mlab-2.3.2/OUTPUT/results.xls", sheetname = "all IRFs Mon. Pol. Shock")

In [167]:
all_irf = old_irf_df.T
new_header = all_irf.iloc[0] # grab the first row for the header
all_irf = all_irf[1:] # take the data less the header row
stripped_headers = [myHeader.strip() for myHeader in np.array(new_header)] # removing trailing whitespaces
all_irf.columns = stripped_headers # set the header row as the df header
all_irf["c_t"] = all_irf.index
all_irf.index = np.arange(0,21,1)
all_irf.index.name = "Period"

# This section rearranges the columns
n = len(list(df.columns.values))
rearranged = [list(all_irf.columns.values)[-1]] + list(all_irf.columns.values)[:n-1]
all_irf = all_irf[rearranged]
all_irf.head()

,c_t,wtilde_t,lambda_zstar_t,m_t,pi_t,x_t,s_t,i_t,h_t,kbar_t1,...,AUX_ENDO_LAG_69_3,AUX_ENDO_LAG_68_1,AUX_ENDO_LAG_68_2,AUX_ENDO_LAG_68_3,AUX_ENDO_LAG_65_1,AUX_ENDO_LAG_65_2,AUX_ENDO_LAG_65_3,AUX_ENDO_LAG_67_1,AUX_ENDO_LAG_67_2,AUX_ENDO_LAG_67_3
Period,,,,,,,,,,,,,,,,,,,,,
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,-0.0307227,0.00011816,1.17882e-15,-2.63252e-17,0.00545838,-0.576329,0.271231,7.25589e-17,0.0481647,2.09068e-16,...,0,0,0,0,0,0,0,0,0,0
2,-2.71466e-18,-1.81896e-16,6.61791e-16,-0.576329,-1.18572e-16,0.576329,2.56694e-17,3.03174e-16,-1.34159e-16,4.56395e-16,...,0,0.0311337,0,0,1.01557,0,0,6.54954e-16,9.01743e-16,-1.67652e-18
3,-5.00618e-17,-1.36367e-16,-6.10155e-16,-7.84889e-17,-1.64344e-16,-9.45684e-16,-2.13683e-16,6.73099e-16,-6.26467e-16,9.77542e-16,...,0,2.18432e-16,0.0311337,0,3.70086e-16,1.01557,0,-6.51354e-16,4.02724e-16,9.07196e-16
4,-7.1145e-17,7.83955e-17,-6.80837e-17,-9.42348e-16,-2.39539e-18,7.31034e-16,-2.84422e-16,1.97296e-16,-6.11136e-16,1.07054e-15,...,0.0311337,2.22032e-16,2.18432e-16,0.0311337,8.16841e-16,3.70086e-16,1.01557,2.49008e-16,-6.51354e-16,4.02724e-16


### Here, we import the data for 4 ACF (Autocorrelation Function) variables
#### outputgap, inflation, interest, output

In [182]:
acf = pd.read_excel("../mmb-gui-mlab-2.3.2/OUTPUT/results.xls", sheetname = "ACF")
acf = acf.T
acf_headers = acf.iloc[0] # grab the first row for the header
acf = acf[1:] # take the data less the header row
acf_stripped_headers = [myHeader.strip() for myHeader in np.array(acf_headers)] # removing trailing whitespaces
acf.columns = acf_stripped_headers
acf = acf.iloc[:, [i for i in range(0, len(acf.columns.values), 2)]]
acf.columns = ["OutputGap", "Inflation", "Interest", "Output"]
acf = acf.reset_index()
acf.index.name = "Period"
acf.drop('index', axis=1, inplace=True)
acf.head()

,OutputGap,Inflation,Interest,Output
Period,,,,
0,1,1,1,1
1,0.99073,0.992913,0.991293,0.990932
2,0.973591,0.972708,0.969202,0.971704
3,0.951359,0.94178,0.936439,0.94537
4,0.925587,0.90317,0.896073,0.91379
